In [1]:
import os
from google.colab import drive
drive.mount('/content/drive', force_remount=True)
os.chdir('/content/drive/MyDrive/surya/')

Mounted at /content/drive


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
!pip install transformers faiss-cpu PyMuPDF pdf2image pillow
!apt-get install -y poppler-utils


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.5/27.5 MB 63.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.6/19.6 MB 75.4 MB/s eta 0:00:00
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following NEW packages will be installed:
  poppler-utils
0 upgraded, 1 newly installed, 0 to remove and 49 not upgraded.
Need to get 186 kB of archives.
After this operation, 696 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 poppler-utils amd64 22.02.0-2ubuntu0.5 [186 kB]
Fetched 186 kB in 1s (183 kB/s)
Selecting previously unselected package poppler-utils.
(Reading database ... 123622 files and directories currently installed.)
Preparing to unpack .../poppler-utils_22.02.0-2ubuntu0.5_amd64.deb ...
Unpacking poppler-utils (22.02.0-2ubuntu0.5) ...
Setting up poppler-utils (22.02.0-2ubuntu0.5) ...
Processing triggers for man-db (2.10.2-1) ...


In [4]:
import fitz  # PyMuPDF for PDF text extraction
from pdf2image import convert_from_path
from PIL import Image

# Path to your PDF file (upload to Colab first)
pdf_path = "/content/drive/MyDrive/surya/Comprehensive_Stock_Market_Data.pdf"

# Initialize lists to hold text and image data
pdf_texts = []
pdf_images = []

# Extract text and images from each page
with fitz.open(pdf_path) as pdf:
    for page_num in range(pdf.page_count):
        page = pdf[page_num]
        pdf_texts.append(page.get_text())  # Extract text from page

        # Convert page to an image
        images = convert_from_path(pdf_path, first_page=page_num + 1, last_page=page_num + 1)
        pdf_images.extend(images)  # Store images


In [5]:
from transformers import CLIPProcessor, CLIPModel
import torch
import numpy as np

# Initialize CLIP model and processor for embeddings
clip_model = CLIPModel.from_pretrained("openai/clip-vit-base-patch32")
processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")

# Function to split long text into chunks that fit the model's limit
def split_text(text, chunk_size=77):
    words = text.split()
    return [' '.join(words[i:i + chunk_size]) for i in range(0, len(words), chunk_size)]

# Prepare text data by splitting long texts into chunks of 77 tokens or less
chunked_texts = []
for text in pdf_texts:  # pdf_texts contains extracted text from the PDF
    chunked_texts.extend(split_text(text))

# Generate embeddings for each text chunk
text_inputs = processor(text=chunked_texts, return_tensors="pt", padding=True, truncation=True, max_length=77)
text_embeddings = clip_model.get_text_features(**text_inputs)
text_embeddings = text_embeddings / text_embeddings.norm(dim=-1, keepdim=True)  # Normalize embeddings

# Process and generate image embeddings
image_inputs = processor(images=pdf_images, return_tensors="pt")  # pdf_images contains images from PDF
image_embeddings = clip_model.get_image_features(**image_inputs)
image_embeddings = image_embeddings / image_embeddings.norm(dim=-1, keepdim=True)  # Normalize embeddings

# Convert embeddings to numpy arrays for FAISS compatibility
text_embeddings = text_embeddings.detach().cpu().numpy()
image_embeddings = image_embeddings.detach().cpu().numpy()


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/4.19k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/605M [00:00<?, ?B/s]

preprocessor_config.json:   0%|          | 0.00/316 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/592 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/862k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/525k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.22M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/389 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [6]:
!pip install langchain
!pip install -U langchain-community


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.6/50.6 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 21.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 408.0/408.0 kB 23.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 296.9/296.9 kB 20.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.0/78.0 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 144.5/144.5 kB 10.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.5/54.5 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 41.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 3.4 MB/s eta 0:00:00


In [7]:
from langchain.vectorstores import FAISS
import numpy as np

# Combine text and image embeddings and metadata
combined_texts = chunked_texts + ["Image"] * len(pdf_images)  # Text labels for each embedding
combined_embeddings = np.concatenate((text_embeddings, image_embeddings), axis=0)  # Combined embeddings

# Initialize FAISS with precomputed embeddings and metadata
vector_store = FAISS.from_embeddings(
    text_embeddings=[(text, embedding) for text, embedding in zip(combined_texts, combined_embeddings)],
    embedding=combined_embeddings
)

print("FAISS vector store created successfully.")


FAISS vector store created successfully.


In [8]:
import torch

# Function to generate embedding for a query
def generate_text_embedding(query):
    text_input = processor(text=query, return_tensors="pt", padding=True, truncation=True, max_length=77)
    text_embedding = clip_model.get_text_features(**text_input)
    text_embedding = text_embedding / text_embedding.norm(dim=-1, keepdim=True)  # Normalize embedding
    return text_embedding.detach().cpu().numpy()

def generate_image_embedding(image):
    image_input = processor(images=image, return_tensors="pt")
    image_embedding = clip_model.get_image_features(**image_input)
    image_embedding = image_embedding / image_embedding.norm(dim=-1, keepdim=True)  # Normalize embedding
    return image_embedding.detach().cpu().numpy()

#General Performance Questions:

In [10]:
# Example 1: Text Query
text_query = "What was the closing price of Apple Inc. in Q1 2024?"  # Example query text
text_query_embedding = generate_text_embedding(text_query)
text_results = vector_store.similarity_search_by_vector(text_query_embedding[0], k=1)  # Retrieve top 5 matches
print("Text Query Results:")
for result in text_results:
    print(result.page_content)  # Display the retrieved text descriptions


Text Query Results:
Stock Market Overview (2023 - 2024) 1. Apple Inc. (AAPL) - Q1 2023: - Closing Price: $170.00 - Market Cap: $2.7 Trillion - Earnings Per Share (EPS): $5.67 - Q2 2023: - Closing Price: $160.00 - Market Cap: $2.5 Trillion - Earnings Per Share (EPS): $4.95 - Q3 2023: - Closing Price: $155.00 - Market Cap: $2.45 Trillion - Earnings Per Share (EPS): $4.80 - Q4 2023: - Closing Price: $165.00 - Market Cap: $2.6 Trillion -


In [11]:
# Example 2: Text Query
text_query = "Which company had the highest EPS in Q4 2023?"  # Example query text
text_query_embedding = generate_text_embedding(text_query)
text_results = vector_store.similarity_search_by_vector(text_query_embedding[0], k=1)  # Retrieve top 5 matches
print("Text Query Results:")
for result in text_results:
    print(result.page_content)  # Display the retrieved text descriptions

Text Query Results:
Market Cap: $2.5 Trillion - Earnings Per Share (EPS): $8.80 3. Amazon.com, Inc. (AMZN) - Q1 2023: - Closing Price: $3,000.00 - Market Cap: $1.5 Trillion - Earnings Per Share (EPS): $10.50 - Q2 2023:


#Trends and Insights

In [12]:
# Example 3: Text Query
text_query = "What trends can be observed in the tech sector over the quarters?"  # Example query text
text_query_embedding = generate_text_embedding(text_query)
text_results = vector_store.similarity_search_by_vector(text_query_embedding[0], k=1)  # Retrieve top 5 matches
print("Text Query Results:")
for result in text_results:
    print(result.page_content)  # Display the retrieved text descriptions

Text Query Results:
- Closing Price: $860.00 - Market Cap: $790 Billion - Earnings Per Share (EPS): $2.70 - Q1 2024: - Closing Price: $940.00 - Market Cap: $850 Billion - Earnings Per Share (EPS): $3.10 6. Meta Platforms, Inc. (META) - Q1 2023: - Closing Price: $350.00 - Market Cap: $950 Billion - Earnings Per Share (EPS): $12.00 - Q2 2023: - Closing Price: $320.00 - Market Cap: $900 Billion - Earnings Per Share (EPS): $11.50 - Q3 2023:


In [ ]:
# Example 2: Image Query
image_query = pdf_images[0]  # Example image from PDF
image_query_embedding = generate_image_embedding(image_query)
image_results = vector_store.similarity_search_by_vector(image_query_embedding[0], k=5)  # Retrieve top 5 matches
print("\nImage Query Results:")
for result in image_results:
    print(result.page_content)

In [ ]:
# Function to generate an embedding for the text query
def generate_text_embedding(query):
    text_input = processor(text=query, return_tensors="pt", padding=True, truncation=True, max_length=77)
    text_embedding = clip_model.get_text_features(**text_input)
    text_embedding = text_embedding / text_embedding.norm(dim=-1, keepdim=True)  # Normalize embedding
    return text_embedding.detach().cpu().numpy()

# Example text query asking for an image
text_query = "Provide me an image of biodata."  # Example text query
text_query_embedding = generate_text_embedding(text_query)

# Search the vector store using the text embedding
results = vector_store.similarity_search_by_vector(text_query_embedding[0], k=10)  # Retrieve top 10 matches

print("\nResults for Query About an Image:")
for result in results:
    # Check if the result is an image based on metadata
    if result.metadata.get("type") == "image":
        display_image(result.page_content)  # Replace with actual display logic for the image
    else:
        print("Description:", result.page_content)  # Show text description for non-image results
